Import all necessary modules

In [ ]:
# imports: this cell will have all the imports used in this notebook
import random
import time
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

from scipy import stats
from scipy.stats import bernoulli
from bitstring import BitArray

# sklearn imports
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split as split
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# sklearn genetics
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Integer, Categorical, Continuous
from sklearn_genetic.plots import plot_fitness_evolution, plot_search_space
from sklearn_genetic.callbacks import LogbookSaver, ProgressBar

In [ ]:
# Sklearn
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor

# importing the minmaxscaler to normalize data between 0 and 1
from sklearn.preprocessing import MinMaxScaler

# keres imports
from keras.layers import LSTM, Input, Dense, Dropout, Activation
from keras.models import Model
from keras.models import Sequential

# Deap for genetic algorithm imports
from deap import base, creator, tools, algorithms

# seeding to get reproducible results with Keras and numpy
from numpy.random import seed
import tensorflow

# used for displaying missing data
import missingno as msno

%matplotlib inline
from subprocess import check_output
%config Completer.use_jedi = False

In [ ]:

# preprocessing 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF,  DotProduct, ConstantKernel as C
from sklearn.linear_model import BayesianRidge
from sklearn.neural_network import MLPRegressor
from scipy.stats import loguniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import  mean_squared_error, r2_score, make_scorer

seed(1)
tensorflow.random.set_seed(2)

%matplotlib inline

Set image printing features

In [ ]:
# Use TeX fonts
plt.rc("text", usetex=False)
plt.rc('font', **{'family': 'serif', 'serif': ['cmr10']})
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rc('font', size=7.0)
plt.rc('font', weight='normal')
plt.rc('legend', fontsize=5)
plt.rc('axes', grid=False)
plt.rcParams['axes.labelsize'] = 7
plt.rcParams['xtick.labelsize'] = 5
plt.rcParams['ytick.labelsize'] = 5
plt.rcParams['figure.figsize'] = [11, 6.5]

Define input and output dataframes and view them

In [ ]:
# read and convert csv data into a dataframe and print it out
df_inp = pd.read_csv("inputs_sample_m.csv")
df_inp = df_inp.dropna()
df_inp.head(n=10)

In [ ]:
df_out = pd.read_csv("output_sample_m.csv")
df_out = df_out.dropna()
df_out.head(n=10)

In [ ]:
lst = list(df_inp.columns)
df_inp[lst].hist(figsize=(15,10))

Correlation 

In [ ]:
corrMatrix = df_inp.corr()
print(corrMatrix)

In [ ]:
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
# finding correlation between the inputs and outputs
corrl = df_inp.corrwith(df_out["H2"])   # we first find for Hydrogen production
corr2 = df_inp.corrwith(df_out["Temp"])  # we then find for Temprature
corr3 = df_inp.corrwith(df_out["Press"])   # we then find for Pressure
print("Correlation of the input data with Hydrogen production")
print(corrl)
print("\n")
print("Correlation of the input data with Temprature")
print(corr2)
print("\n")
print("Correlation of the input data with Pressure")
print(corr3)

Ploting the pearson's correlation in a horizontal bar graph between the inputs and each output

In [ ]:
# Now plotting Pearson's r between inputs and outputs
inputs_or = df_inp.columns.to_list()

plt.barh(inputs_or,corrl,color='blue') # blue for plot for hydrogen production

In [ ]:
plt.barh(inputs_or,corr2,color='red') # red for plot for Tempreature

In [ ]:
plt.barh(inputs_or,corr3,color='green') # green for plot for pressure

Calculating the Spearman's P Values between the inputs and each output

In [ ]:
# find the p-value of the data and print it out
pph_val = []
for i in inputs_or:
    pcor, pval = stats.spearmanr(df_inp[i],df_out["H2"])   # we first find for Hydrogen production
    pph_val.append(pval)
print(pph_val)

In [ ]:
plt.barh(inputs_or,pph_val,color='blue') # blue for p-values of input with hydrogen production
plt.axvline(x=0.05,color='black')

In [ ]:
# find the p-value of the data and print it out
ppt_val = []
for i in inputs_or:
    pcor, pval = stats.spearmanr(df_inp[i],df_out["Temp"])   # then find for temperature
    ppt_val.append(pval)
print(ppt_val)

In [ ]:
plt.barh(inputs_or,ppt_val,color='red') # red for p-values of input with Tempreature
plt.axvline(x=0.05,color='black')

In [ ]:
# find the p-value of the data and print it out
ppp_val = []
for i in inputs_or:
    pcor, pval = stats.spearmanr(df_inp[i],df_out["Press"])   # then find for pressure
    ppp_val.append(pval)
print(ppp_val)

In [ ]:
plt.barh(inputs_or,ppp_val,color='green') # green for p-values of input with pressure
plt.axvline(x=0.05,color='black')

Base Graphs

In [ ]:
dfb = pd.read_csv("base_sample.csv")
dfb

In [ ]:
dfb.plot(x='Time', y='H2')
plt.xlabel("Time/ hours")
plt.ylabel("Hydrogen/ quantity generated")

In [ ]:
dfb.plot(x='Time', y='Press')
plt.xlabel("Time/ hours")
plt.ylabel("Pressure/ Pa")

In [ ]:
dfb.plot(x='Time', y='Temp')
plt.xlabel("Time/ hours")
plt.ylabel("Tempreature/ K")

Data cleaning process

In [ ]:
# set values of input to x
X = df_inp.values

# set values of output to y_out
y_opt = df_out.values

In [ ]:
# build a pipeline for scaling and 
distributions = [
    ('Unscaled data', X),
    ('Data after standard scaling',
        StandardScaler().fit_transform(X)),
    ('Data after min-max scaling',
        MinMaxScaler().fit_transform(X)),
    ('Data after max-abs scaling',
        MaxAbsScaler().fit_transform(X)),
    ('Data after robust scaling',
        RobustScaler(quantile_range=(25, 75)).fit_transform(X)),
    ('Data after power transformation (Yeo-Johnson)',
     PowerTransformer(method='yeo-johnson').fit_transform(X)),
#     ('Data after power transformation (Box-Cox)',
#      PowerTransformer(method='box-cox').fit_transform(X)),
    ('Data after quantile transformation (uniform pdf)',
        QuantileTransformer(output_distribution='uniform')
        .fit_transform(X)),
    ('Data after quantile transformation (gaussian pdf)',
        QuantileTransformer(output_distribution='normal')
        .fit_transform(X)),
    ('Data after sample-wise L2 normalizing',
        Normalizer().fit_transform(X)),
]

# scale the output between 0 and 1 for the colorbar
y = minmax_scale(y_opt)

# X_train, X_test, y_train, y_test = data_preprocess(X,y)
item_idx = 1 
title, X = distributions[item_idx]
print(title)

In [ ]:
# visualize outliers
sn.boxplot( data=X)

In [ ]:
# visualize outliers
sn.boxplot( data=y)

In [ ]:
X_train, X_test, y_train, y_test = split(X, y, test_size=0.30)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Using GA

In [29]:
# Define the parameter grid for the RandomForestRegressor
pipe = RandomForestRegressor()

param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30), 
              'max_leaf_nodes': Integer(2, 35), 
              'n_estimators': Integer(100, 300)}

# Define the GASearchCV options
evolved_estimator = GASearchCV(
    estimator=pipe,
    cv=10,
    scoring="r2",
    population_size=15,
    generations=20,
    tournament_size=3,
    elitism=True,
    keep_top_k=4,
    crossover_probability=0.9,
    mutation_probability=0.05,
    param_grid=param_grid,
    criteria="max",
    algorithm="eaMuCommaLambda",
    n_jobs=-1)

#Optionally, create some Callbacks
callbacks = [LogbookSaver(checkpoint_path="./logbook.pkl"), ProgressBar()]

# Fit the model and see some results
evolved_estimator.fit(X_train, y_train, callbacks=callbacks)
y_predict_ga = evolved_estimator.predict(X_test)
r_squared = r2_score(y_test, y_predict_ga)

print(evolved_estimator.best_params_)
print("r-squared: ", "{:.2f}".format(r_squared))
print("Best k solutions: ", evolved_estimator.hof)

In [ ]:
plot = plot_fitness_evolution(evolved_estimator, metric="fitness")

In [ ]:
plt.rc("text", usetex=False)
plot_search_space(evolved_estimator)

In [ ]:
plt.figure(figsize=[6,5])
plt.scatter(y_test, y_predict_ga, s=90)
# plt.xlim(left=0.86)
# plt.ylim(bottom=0.88, top=0.96)

In [ ]:
def plot_model_performance(model_name, y_test, y_predict):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[8,5], dpi=80)

    fig.suptitle(f'Model Performance of {model_name}')
    ax1.scatter(y_test, y_predict, s=120, alpha=0.5)
    ax1.set_xlim(left=min(y_test))
    ax1.set_ylim(bottom=min(y_predict), top=max(y_predict))
    ax1.set_xlabel(r'Scaled Actual Output')
    ax1.set_ylabel(r'Scaled Predicted Output')
    ax1.set_title(r'$R^2 = {:.4g}$'.format(r2_score(y_test, y_predict)))

    # best fit of data
    data_res = y_test - y_predict 
    (mu, sigma) = stats.norm.fit(data_res[data_res > min(data_res)])

    # the histogram of the data
    n, bins, patches = ax2.hist(y_test - y_predict, 30, density=1, alpha=0.5)

    # add a 'best fit' line
    best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
    l = ax2.plot(bins, best_fit_line, 'r--', linewidth=2)

    ax2.set_xlim(left=min(data_res))
    ax2.set_xlabel(r'Residual')
    ax2.set_ylabel(r'Frequency')
    ax2.set_title(r'$\sigma = {:.4f}$'.format(mean_squared_error(y_test, y_predict, squared=False)))

    fig.tight_layout()

    plt.show()

In [ ]:
model_name = "Hydrogen"
plot_model_performance(model_name, y_test[:,0], y_predict_ga[:,0])

In [ ]:
model_name = "Temperature"
plot_model_performance(model_name, y_test[:,1], y_predict_ga[:,1])

In [ ]:
model_name = "Pressure"
plot_model_performance(model_name, y_test[:,2], y_predict_ga[:,2])

MLP

In [ ]:
pipe = MLPRegressor()

param_grid = {
#     'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
#               'bootstrap': Categorical([True, False]),
#               'max_depth': Integer(2, 30), 
#               'max_leaf_nodes': Integer(2, 35), 
#               'n_estimators': Integer(100, 300)             
          'solver': Categorical(['lbfgs', 'sgd', 'adam']),
          'max_iter': Integer(500,1500),
          'alpha': Continuous(10.0**(-7), 10.0**(-1), distribution='log-uniform'),
          'hidden_layer_sizes': Integer(5, 12),
          'random_state':Integer(0, 9)
             }

# Define the GASearchCV options
evolved_estimator = GASearchCV(
    estimator=pipe,
    cv=10,
    scoring="r2",
    population_size=15,
    generations=20,
    tournament_size=3,
    elitism=True,
    keep_top_k=4,
    crossover_probability=0.9,
    mutation_probability=0.05,
    param_grid=param_grid,
    criteria="max",
    algorithm="eaMuCommaLambda",
    n_jobs=-1)

#Optionally, create some Callbacks
callbacks = [LogbookSaver(checkpoint_path="./logbook.pkl"), ProgressBar()]

# Fit the model and see some results
evolved_estimator.fit(X_train, y_train, callbacks=callbacks)
y_predict_ga = evolved_estimator.predict(X_test)
r_squared = r2_score(y_test, y_predict_ga)

print(evolved_estimator.best_params_)
print("r-squared: ", "{:.2f}".format(r_squared))
print("Best k solutions: ", evolved_estimator.hof)

In [ ]:
plot = plot_fitness_evolution(evolved_estimator, metric="fitness")

In [ ]:
plt.rc("text", usetex=False)
plot_search_space(evolved_estimator)

In [ ]:
plt.figure(figsize=[6,5])
plt.scatter(y_test, y_predict_ga, s=90)
# plt.xlim(left=0.86)
# plt.ylim(bottom=0.88, top=0.96)

In [ ]:
model_name = "Hydrogen"
plot_model_performance(model_name, y_test[:,0], y_predict_ga[:,0])

In [ ]:
model_name = "Temperature"
plot_model_performance(model_name, y_test[:,1], y_predict_ga[:,1])

In [ ]:
model_name = "Pressure"
plot_model_performance(model_name, y_test[:,2], y_predict_ga[:,2])

In [ ]:
# Reshape the target variables
# y_train = np.reshape(y_train, (-1,))
# y_test = np.reshape(y_test, (-1,))

# # Define a custom scoring metric for R-squared
# def r2_score_3d(y_true, y_pred):
#     y_true = np.reshape(y_true, (-1,))
#     y_pred = np.reshape(y_pred, (-1,))
#     return r2_score(y_true, y_pred)

# scoring_metric = make_scorer(r2_score_3d)

# param_grid = {'n_iter': range(300, 501),
#               'tol': np.linspace(0.0001, 0.1, 100),
#               'alpha_1': np.linspace(1.0e-7, 1e-1, 100),
#               'alpha_2': np.linspace(1.0e-7, 1e-1, 100),
#               'lambda_1': np.linspace(1.0e-7, 1e-1, 100),
#               'lambda_2': np.linspace(1.0e-7, 1e-1, 100),
#               'normalize': [False, True]
#               }

# pipe = BayesianRidge()


# # Create the CV strategy
# cv = KFold(n_splits=10, shuffle=True)

# # Create the EvolutionaryAlgorithmSearchCV object
# evolved_estimator = GASearchCV(
#     estimator=pipe,
#     param_grid=param_grid,
#     scoring=scoring_metric,
#     cv=cv,
#     population_size=50,
#     generations=30,
#     tournament_size=3,
#     elitism=True,
#     keep_top_k=4,
#     crossover_probability=0.9,
#     mutation_probability=0.05,
#     verbose=True,
#     criteria="max",
#     algorithm="eaMuCommaLambda",
#     n_jobs=-1
# )

# #Optionally, create some Callbacks
# callbacks = [LogbookSaver(checkpoint_path="./logbook.pkl"), ProgressBar()]

# # Fit the model and see some results
# evolved_estimator.fit(X_train, y_train, callbacks=callbacks)
# y_predict_ga = evolved_estimator.predict(X_test)
# y_predict_ga = np.reshape(y_predict_ga, y_test.shape)
# r_squared = r2_score_3d(y_test, y_predict_ga)

# print(evolved_estimator.best_params_)
# print("r-squared: ", "{:.2f}".format(r_squared))
# print("Best k solutions: ", evolved_estimator.hof)

param_grid = {'n_iter': Integer(300, 500),
              'tol': Continuous(0.0001, 0.1),
              'alpha_1': Continuous(1.0e-7, 1e-1),
              'alpha_2': Continuous(1.0e-7, 1e-1),
              'lambda_1': Continuous(1.0e-7, 1e-1),
              'lambda_2': Continuous(1.0e-7, 1e-1),
              'normalize': Categorical([False, True])}



pipe = BayesianRidge()

# param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
#               'bootstrap': Categorical([True, False]),
#               'max_depth': Integer(2, 30), 
#               'max_leaf_nodes': Integer(2, 35), 
#               'n_estimators': Integer(100, 300)}
# Create the CV strategy and define the param grid
cv = KFold(n_splits=10, shuffle=True)

# Define the GASearchCV options
evolved_estimator = GASearchCV(
    estimator=pipe,
    cv=cv,
    scoring="r2",
    population_size=15,
    generations=20,
    tournament_size=3,
    elitism=True,
    keep_top_k=4,
    crossover_probability=0.9,
    mutation_probability=0.05,
    param_grid=param_grid,
#     param_grid=None,
    criteria="max",
    algorithm="eaMuCommaLambda",
    n_jobs=-1)

#Optionally, create some Callbacks
callbacks = [LogbookSaver(checkpoint_path="./logbook.pkl"), ProgressBar()]

# Fit the model and see some results
evolved_estimator.fit(X_train, y_train, callbacks=callbacks)
y_predict_ga = evolved_estimator.predict(X_test)
r_squared = r2_score(y_test, y_predict_ga)

print(evolved_estimator.best_params_)
print("r-squared: ", "{:.2f}".format(r_squared))
print("Best k solutions: ", evolved_estimator.hof)

In [ ]:
plot = plot_fitness_evolution(evolved_estimator, metric="fitness")

In [ ]:
plt.rc("text", usetex=False)
plot_search_space(evolved_estimator)

In [ ]:
plt.figure(figsize=[6,5])
plt.scatter(y_test, y_predict_ga, s=90)
# plt.xlim(left=0.86)
# plt.ylim(bottom=0.88, top=0.96)

In [ ]:
model_name = "Hydrogen"
plot_model_performance(model_name, y_test[:,0], y_predict_ga[:,0])

In [ ]:
model_name = "Temperature"
plot_model_performance(model_name, y_test[:,1], y_predict_ga[:,1])

In [ ]:
model_name = "Pressure"
plot_model_performance(model_name, y_test[:,2], y_predict_ga[:,2])